In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

In [2]:
# Testing load graph from json
deep_reload(src)

filename = 'graph.json'

graph = src.graph.graph_from_json(filename)

In [273]:
import json

filename = 'parameters_cec_router.json'

with open(filename, 'r') as file:

    parameters = json.load(file)

In [235]:
# Assigning nodes to depots based on Voronoi cells
deep_reload(src)

center_nodes = parameters['depot_nodes']
weight = parameters['voronoi_weight']

graph = src.router.assign_depot(graph, center_nodes, weight = weight)

In [236]:
# Assinging random number to each node for selection
deep_reload(src)

seed = parameters['rng_seed']

graph = src.router.assign_rng(graph, seed)

In [237]:
# Assinging vehicles to nodes
deep_reload(src)

vehicles = parameters['vehicles']

graph = src.router.assign_vehicle(graph, vehicles)

In [238]:
# Producing subgraphs for routing
deep_reload(src)

categories = {
    'vehicle': list(parameters['vehicles'].keys()),
    'depot': parameters['depot_nodes'],
}

subgraphs = src.router.produce_subgraphs(graph, categories)
# subgraphs

In [240]:
# Producing adjacency matrices for routing
deep_reload(src)

weights = parameters['route_weights']

adjacency = src.router.produce_adjacency(subgraphs, weights)

In [11]:
# Producing assignments or routing
deep_reload(src)

assignments = src.router.produce_assignments(subgraphs)

In [249]:
# Producing bounds
deep_reload(src)

vehicles = parameters['vehicles']
weights = parameters['route_weights']

route_bounds, leg_bounds, stop_weights = src.router.produce_bounds(
    subgraphs, vehicles, weights)

In [254]:
vehicle = 'Bolt (Short)'
depot = 1000000000

case = (vehicle, depot)

route_bounds[case]

[[0, 400000.0], [0, 10800]]

In [274]:
# Producing routing inputs
deep_reload(src)

cases = src.router.produce_routing_inputs(graph, parameters)

In [276]:
# Producing un-optimized routes
deep_reload(src)

vehicle = 'Bolt (Infinite)'
depot = 1000000000

case = cases[(vehicle, depot)]

routes, success = src.router.depot_router(
    case['adjacency'],
    case['assignments']['node_to_idx'][depot],
    case['route_bounds'],
    case['leg_bounds'],
    case['stop_weights'],
)

In [152]:
# Producing un-optimized routes
deep_reload(src)

combination = ('Bolt', 1000000000)
case_index = 1
route_bounds_case = [rb[case_index] for rb in route_bounds[combination]]
leg_bounds_case = [rb[case_index] for rb in leg_bounds[combination]]
stop_weights_case = [rb[case_index] for rb in stop_weights[combination]]

routes, success = src.router.clarke_wright(
    adjacency[combination],
    assignments[combination]['node_to_idx'][combination[1]],
    route_bounds_case,
    leg_bounds_case,
    stop_weights_case,
)

In [25]:
deep_reload(src)

weights, validity = src.router.evaluate_route(
    adjacency[combination],
    routes[8],
    route_bounds_case,
    leg_bounds_case,
    stop_weights_case,
)

weights[0], weights[1]/3600, validity

(399570.0, 7.506465749528248, [True, True])

In [27]:
deep_reload(src)

weights, validity = src.router.evaluate_routes(
    adjacency[combination],
    routes,
    route_bounds_case,
    leg_bounds_case,
    stop_weights_case,
)

In [80]:
deep_reload(src)

opt_route = src.router.anneal_route(
    adjacency[combination],
    routes[5],
    route_bounds_case,
    leg_bounds_case,
    stop_weights_case,
)

[192950.0, 23015.310052447552] [242280.0, 25344.692794705294]


In [77]:
opt_route

[213, 3, 27, 4, 92, 174, 213]

In [214]:
deep_reload(src)

opt_routes = src.router.anneal_routes(
    adjacency[combination],
    routes,
    route_bounds_case,
    leg_bounds_case,
    stop_weights_case,
    steps = 1000,
)

27714450.0 27336370.0


In [271]:
import json
json.loads('[Infinity]')[0]+1

inf

In [268]:
json.loads('[Infinity, -Infinity, NaN]')

[inf, -inf, nan]